# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 9 Assignment: Kaggle Submission**

**Student Name: Julia Huang**

# Assignment Instructions

For this assignment you will begin by loading a pretrained neural network that I provide here:  [transfer_9.h5](https://data.heatonresearch.com/data/t81-558/networks/transfer_9.h5).  You will demonstrate your ability to transfer several layers from this neural network to create a new neural network to be used for feature engineering.

The **transfer_9.h5** neural network is composed of the following four layers:

```
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_11 (Dense)             (None, 25)                225       
_________________________________________________________________
dense_12 (Dense)             (None, 10)                260       
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 33        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 4         
=================================================================
Total params: 522
Trainable params: 522
Non-trainable params: 0
```

You should only use the first three layers.  The final dense layer should be removed, exposing the (None, 3) shaped layer as the new output layer.  This is a 3-neuron layer.  The output from these 3 layers will become your 3 engineered features.
  
Complete the following tasks:

* Load the Keras neural network **transfer_9.h5**.  Note that you will need to download it to either your hard drive or GDrive (if you're using Google CoLab).  Keras does not allow loading of a neural network across HTTP.
* Create a new neural network with only the first 3 layers, drop the (None, 1) shaped layer.
* Load the dataset [transfer_data.csv](https://data.heatonresearch.com/data/t81-558/datasets/transfer_data.csv). 
* Use all columns as input, but do not use *id* as input.  You will need to save the *id* column to build your submission.
* Do not zscore or transform the input columns.
* Submit the output from the (None, 3) shaped layer, along with the corresponding *id* column.  The three output neurons should create columns named *a*, *b*, and *c*.

The submit file will look something like:

|id|a|b|c|
|-|-|-|-|
|1|2.3602087|1.4411213|0|
|2|0.067718446|1.0037427|0.52129996|
|3|0.74778837|1.0647631|0.052594826|
|4|1.0594225|1.1211816|0|
|...|...|...|...|




# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process.  Running the following code will map your GDrive to /content/drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls /content/drive/My\ Drive/Colab\ Notebooks

assignment_hjulia_class7.ipynb	 assignment_jhuang_class8.ipynb
assignment_jhuang_class10.ipynb  assignment_jhuang_class9.ipynb
assignment_jhuang_class1.ipynb	 assignment_juliahuang_class3.ipynb
assignment_jhuang_class2.ipynb	 Kaggle_features_v1.ipynb
assignment_jhuang_class3.ipynb	 Kaggle_preprocessing_v1.ipynb
assignment_jhuang_class4.ipynb	 Untitled
assignment_jhuang_class5.ipynb	 Untitled0.ipynb
assignment_jhuang_class6.ipynb


# Assignment #9 Sample Code

The following code provides a starting point for this assignment.

In [0]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import load_model
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
import sklearn
from sklearn.linear_model import Lasso

# This is your student key that I emailed to you at the beginnning of the semester.
key = "AriqFC7MCs8Rp5p2GYRK81xa3nCUqQpPawBNrNLi"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/My Drive/Colab Notebooks/assignment_ZachWang_class9.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class9.ipynb'  # Windows
#file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class9.ipynb'  # Mac/Linux

# Begin assignment
model = load_model("/content/drive/My Drive/transfer_9.h5") # modify to where you stored it
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/transfer_data.csv")
df.head()
idx=df.id

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


,id,length,diameter,height,whole,shucked,viscera,shell,rings
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,3,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,4,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,5,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [0]:
model1 = Sequential()
for i in range(3):
    layer = model.layers[i]
    layer.trainable = False
    model1.add(layer)
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 25)                225       
_________________________________________________________________
dense_12 (Dense)             (None, 10)                260       
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 33        
Total params: 518
Trainable params: 0
Non-trainable params: 518
_________________________________________________________________


In [0]:
df.drop('id', axis=1).values

array([[ 0.455 ,  0.365 ,  0.095 , ...,  0.101 ,  0.15  , 15.    ],
       [ 0.35  ,  0.265 ,  0.09  , ...,  0.0485,  0.07  ,  7.    ],
       [ 0.53  ,  0.42  ,  0.135 , ...,  0.1415,  0.21  ,  9.    ],
       ...,
       [ 0.6   ,  0.475 ,  0.205 , ...,  0.2875,  0.308 ,  9.    ],
       [ 0.625 ,  0.485 ,  0.15  , ...,  0.261 ,  0.296 , 10.    ],
       [ 0.71  ,  0.555 ,  0.195 , ...,  0.3765,  0.495 , 12.    ]])

In [0]:
x = df.drop('id', axis=1)
pred=model1.predict(x)
pred
df_submit=pd.DataFrame(pred,columns=("a","b","c"))
df_submit=pd.concat([idx, df_submit], axis=1)
df_submit

,id,a,b,c
0,1,0.0,0.0,0.481899
1,2,0.0,0.0,1.687510
2,3,0.0,0.0,1.099649
3,4,0.0,0.0,0.547262
4,5,0.0,0.0,1.636095
...,...,...,...,...
4172,4173,0.0,0.0,0.236203
4173,4174,0.0,0.0,0.201554
4174,4175,0.0,0.0,0.000000
4175,4176,0.0,0.0,0.018183


In [0]:
df_submit.describe()

,id,a,b,c
count,4177.000000,4177.0,4177.0,4177.000000
mean,2089.000000,0.0,0.0,0.662359
std,1205.940366,0.0,0.0,0.696102
min,1.000000,0.0,0.0,0.000000
25%,1045.000000,0.0,0.0,0.000000
50%,2089.000000,0.0,0.0,0.403117
75%,3133.000000,0.0,0.0,1.259226
max,4177.000000,0.0,0.0,2.392713
